<a href="https://colab.research.google.com/github/Frasierzzz/Project/blob/main/Xsum0_01w_owikifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a9a8fc000bac026f6a3b0e384c06d1ad9ad1c7ecfaf20e1ec1ee0aa3d8faed68
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's depe

In [ ]:
pip install --upgrade datasets evaluate transformers rouge-score fsspec gcsfs

INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2024.10.0
    Uninstalling gcsfs-2024.10.0:
      Successfully uninstalled gcsfs-2024.10.0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import random
from datasets import DatasetDict
from evaluate import load
from datasets import load_dataset

In [ ]:
model_checkpoint = "t5-small"

In [ ]:
raw_datasets = load_dataset("xsum")

README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [ ]:
raw_datasets["validation"][0]

{'document': 'The ex-Reading defender denied fraudulent trading charges relating to the Sodje Sports Foundation - a charity to raise money for Nigerian sport.\nMr Sodje, 37, is jointly charged with elder brothers Efe, 44, Bright, 50 and Stephen, 42.\nAppearing at the Old Bailey earlier, all four denied the offence.\nThe charge relates to offences which allegedly took place between 2008 and 2014.\nSam, from Kent, Efe and Bright, of Greater Manchester, and Stephen, from Bexley, are due to stand trial in July.\nThey were all released on bail.',
 'summary': 'Former Premier League footballer Sam Sodje has appeared in court alongside three brothers accused of charity fraud.',
 'id': '38295789'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
  # ตรวจสอบว่าข้อมูลใน examples["document"] เป็น string เดี่ยว หรือ list ของ string
    if isinstance(examples["document"], str):
        documents = [examples["document"]]  # ถ้าเป็น string เดี่ยว ให้ใส่ลงใน list
    elif isinstance(examples["document"], list):
        documents = examples["document"]  # ถ้าเป็น list อยู่แล้ว ใช้งานได้เลย

    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# สุ่มข้อมูล 10%
random.seed(14)
def sample_10_percent(dataset):
    sample_size = int(len(dataset) * 0.0025)
    indices = random.sample(range(len(dataset)), sample_size)
    return dataset.select(indices)

In [ ]:
sampled_datasets = DatasetDict({
    split: sample_10_percent(dataset) for split, dataset in raw_datasets.items()
})
sampled_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 510
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 28
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 28
    })
})

In [ ]:
test = sampled_datasets ["validation"]
test

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 28
})

In [ ]:
def test_function(text):
  # เตรียมข้อมูลสำหรับการทำนาย
  inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
  # รันโมเดลเพื่อสรุปข้อความ
  outputs = model.generate(inputs, max_length=50, num_beams=4, early_stopping=True)
  # ถอดรหัสผลลัพธ์
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print("Summary:", summary)
  return summary

In [ ]:
test["document"][0]

'The work, originally due to start next month, is now planned to begin in September subject to a procurement process.\n£16.2m has been allocated to Translink for the railway upgrade.\nLast year, Translink said an hourly service on the route was a "priority".\nThe second stage of the upgrade involves major re-signalling between Derry and Coleraine and the introduction of a passing loop.'

In [ ]:
test_function(test["document"][0])

Summary: £16.2m allocated to Translink for the railway upgrade. the second stage involves major re-signalling between Derry and Coleraine.


'£16.2m allocated to Translink for the railway upgrade. the second stage involves major re-signalling between Derry and Coleraine.'

In [ ]:
summarize_with_wikifier(test["document"][0])

'the work, originally due to start next month, is now planned to begin in September. the second stage of the upgrade involves major re-signalling between Derry and Coleraine and the introduction of a passing loop '

In [ ]:
test['summary'][0]

'Work on a second major upgrade to the Londonderry-to-Coleraine railway line has been delayed by four months.'

In [ ]:
test["document"][14]

'Police said it was discovered at Maeve House in the New Lodge area on Monday after a report from a member of the public.\nArmy bomb disposal experts were called to the scene and the suspected explosives taken away.\nThe explosives are said to be enough to make three under-car booby-trap bombs.\nDet Insp Stuart Griffin said considerable inconvenience had been caused to residents and he thanked them for their co-operation.\n"Thankfully these suspected explosives are no longer in the public domain," he added.\nThe find comes two years after a similar discovery at Maeve House.\nTwo and a half kilos of the explosive was found in a cupboard at the building in May 2014.\nSpeaking about the latest seizure, Deputy First Minister Martin McGuinness said he was "hugely relieved and delighted that these explosives have been recovered and made safe".\n"I\'m disappointed but not surprised that there are still people within our society who believe that they have the right to attempt to point us all b

In [ ]:
test_function(test["document"][14])

Summary: the explosives are said to be enough to make three under-car booby-trap bombs. the find comes two years after a similar discovery at Maeve House.


'the explosives are said to be enough to make three under-car booby-trap bombs. the find comes two years after a similar discovery at Maeve House.'

In [ ]:
summarize_with_wikifier(test["document"][14])

'the explosives are said to be enough to make three under-car booby-trap bombs. the explosives are said to be enough to make three under-car booby-trap bombs. the find comes'

In [ ]:
test['summary'][14]

'One and a half kilograms of what is believed to be Semtex plastic explosive have been found at a block of flats in north Belfast.'

In [ ]:
from rouge_score import rouge_scorer

# สร้าง Scorer สำหรับ ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# เก็บผลลัพธ์ ROUGE score
rouge_scores = []

# สมมติว่า test เป็น dataset ที่มี key "text" และ "reference"
for i, data in enumerate(test):  # Loop ข้อมูลใน test
    input_text = data['document']  # ข้อความต้นฉบับ
    reference_summary = data['summary']  # สรุปข้อความที่เป็น reference

    # ใช้ test_function สรุปข้อความ
    generated_summary = test_function(input_text)

    # คำนวณ ROUGE
    score = scorer.score(reference_summary, generated_summary)
    rouge_scores.append(score)
    print(f"ROUGE score for example {i}: {score}")

# สรุปค่าเฉลี่ย ROUGE score
avg_rouge1 = sum([s['rouge1'].fmeasure for s in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([s['rouge2'].fmeasure for s in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([s['rougeL'].fmeasure for s in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")


Summary: £16.2m allocated to Translink for the railway upgrade. the second stage involves major re-signalling between Derry and Coleraine.
ROUGE score for example 0: {'rouge1': Score(precision=0.35, recall=0.3684210526315789, fmeasure=0.358974358974359), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.15, recall=0.15789473684210525, fmeasure=0.15384615384615385)}
Summary: the general penitentiary of Venezuela in central Guarico state was closed in October. the bodies were found during works to refurbish the jail. the number of bodies found since has risen from seven to 14.
ROUGE score for example 1: {'rouge1': Score(precision=0.37142857142857144, recall=0.48148148148148145, fmeasure=0.41935483870967744), 'rouge2': Score(precision=0.17647058823529413, recall=0.23076923076923078, fmeasure=0.20000000000000004), 'rougeL': Score(precision=0.2571428571428571, recall=0.3333333333333333, fmeasure=0.2903225806451613)}
Summary: Richard Gilliland, 64, accused

In [ ]:
!pip install ujson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
import requests
import ujson as json

_WIKIFIER_WIKIFY_URL = "http://www.wikifier.org/annotate-article"
ERROR_KEY = "error"

def get_wikifier_wikify_response(text, api_key, df_ignore=0, words_ignore=0):
    params = {
        "text": text,
        "userKey": api_key,
        "nTopDfValuesToIgnore": df_ignore,
        "nWordsToIgnoreFromList": words_ignore
    }
    r = requests.post(_WIKIFIER_WIKIFY_URL, params)
    if r.status_code == 200:
        resp = json.loads(r.content)
        if ERROR_KEY in resp:
            raise ValueError(f"Error in response: {resp[ERROR_KEY]}")
        return resp
    else:
        raise ValueError(f"HTTP status code 200 expected, got status code {r.status_code} instead")

In [ ]:
def summarize_with_wikifier(text, api_key="pwdxjzlnmegtncwumufswitkbefgxb"):
    # วิเคราะห์ข้อความด้วย Wikifier
    wikifier_response = get_wikifier_wikify_response(text, api_key)
    annotations = [anno['title'] for anno in wikifier_response.get("annotations", [])]
    wikifier_keywords = ", ".join(annotations)

    # รวมคำสำคัญจาก Wikifier กับข้อความต้นฉบับ
    enhanced_text = text + " Keywords: " + wikifier_keywords

    # เตรียมข้อความ
    inputs = tokenizer.encode("summarize: " + enhanced_text, return_tensors="pt", max_length=512, truncation=True)
    # สร้างสรุป
    outputs = model.generate(inputs, max_length=50, num_beams=4, early_stopping=True)
    # ถอดรหัสข้อความ
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
test_function(test["document"][0])

Summary: seventeen people died when gunmen targeted the magazine and a Jewish supermarket. no official events are planned for Thursday, although president Francois Hollande is due to address the police. a second plaque was installed where a police


'seventeen people died when gunmen targeted the magazine and a Jewish supermarket. no official events are planned for Thursday, although president Francois Hollande is due to address the police. a second plaque was installed where a police'

In [ ]:
from rouge_score import rouge_scorer

# สร้าง Scorer สำหรับ ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# เก็บผลลัพธ์ ROUGE score
rouge_scores = []

# สมมติว่า test เป็น dataset ที่มี key "text" และ "reference"
for i, data in enumerate(test):  # Loop ข้อมูลใน test
    input_text = data['document']  # ข้อความต้นฉบับ
    reference_summary = data['summary']  # สรุปข้อความที่เป็น reference

    # ใช้ test_function สรุปข้อความ
    generated_summary = summarize_with_wikifier(input_text)

    # คำนวณ ROUGE
    score = scorer.score(reference_summary, generated_summary)
    rouge_scores.append(score)
    print(f"ROUGE score for example {i}: {score}")

# สรุปค่าเฉลี่ย ROUGE score
avg_rouge1 = sum([s['rouge1'].fmeasure for s in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([s['rouge2'].fmeasure for s in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([s['rougeL'].fmeasure for s in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")


ROUGE score for example 0: {'rouge1': Score(precision=0.2777777777777778, recall=0.5263157894736842, fmeasure=0.36363636363636365), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.1111111111111111, recall=0.21052631578947367, fmeasure=0.14545454545454545)}
ROUGE score for example 1: {'rouge1': Score(precision=0.37142857142857144, recall=0.48148148148148145, fmeasure=0.41935483870967744), 'rouge2': Score(precision=0.17647058823529413, recall=0.23076923076923078, fmeasure=0.20000000000000004), 'rougeL': Score(precision=0.2571428571428571, recall=0.3333333333333333, fmeasure=0.2903225806451613)}
ROUGE score for example 2: {'rouge1': Score(precision=0.1111111111111111, recall=0.2222222222222222, fmeasure=0.14814814814814814), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.08333333333333333, recall=0.16666666666666666, fmeasure=0.1111111111111111)}
ROUGE score for example 3: {'rouge1': Score(precision=0.12, recall=